USECASE-3 ANOMALY DETECTION IN SUPPLY CHAIN- AGENT-3

Pre-requisites:

#This notebook requires permissions to:
- create and delete Amazon IAM roles
- access Amazon Bedrock

#The following access are required for Sagemaker to carry on the usecase:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonBedrockFullAccess
- AmazonDynamoDBFullAccess

EDA and extraction of required columns

In [106]:
import pandas as pd

# Load the dataset
csv_file = r"Supply chain dataset - Supply chain dataset (2).csv"
data = pd.read_csv(csv_file)

def get_anomaly_detection_summary():
    # Filter data to include only rows where 'AnomalyType' is either 'Delay' or 'Unusual Order Volume'
    anomaly_data = data[data['AnomalyType'].isin(['Delay', 'Unusual Order Volume'])]

    # Specify the relevant columns
    relevant_columns = [
        'SupplierName', 'DeliveryTime(days)', 'OrderVolume', 'ReorderQuantity',
        'ReliabilityScore of delivery (percentage)', 'TrafficCondition',
        'AnomalyDetected', 'AnomalyType', 
        'PurchaseHistory', 'DeliveryLocation', 'PromotionalEvents'
    ]
    
    # Extract the relevant columns from the filtered data
    anomaly_data = anomaly_data[relevant_columns]

    # Combine the information into a summary string
    summaries = []
    for _, row in anomaly_data.iterrows():
        summary = (
            f"The product {row['PurchaseHistory']} with volume {row['OrderVolume']} "
            f"was delivered to {row['DeliveryLocation']} after {row['DeliveryTime(days)']} days. "
            f"SupplierName: {row['SupplierName']} with reliability score {row['ReliabilityScore of delivery (percentage)']}. "
            f"Traffic condition: {row['TrafficCondition']}. "
            f"Reorder quantity: {row['ReorderQuantity']}. Promotional events: {row['PromotionalEvents']}. "
            f"Anomaly detected: {row['AnomalyDetected']} (Type: {row['AnomalyType']})."
        )
        summaries.append(summary)
    
    result = " ".join(summaries)
    return result

# Get the summary for anomaly detection
summary = get_anomaly_detection_summary()

# Print the summary
print(summary)


The product Smartphone with volume 25 was delivered to Mumbai after 11 days. SupplierName: Supplier_C with reliability score 81. Traffic condition: Normal. Reorder quantity: 22. Promotional events: Event_2. Anomaly detected: True (Type: Delay). The product Keyboard with volume 14 was delivered to Jaipur after 8 days. SupplierName: Supplier_E with reliability score 63. Traffic condition: Congested. Reorder quantity: 70. Promotional events: Event_2. Anomaly detected: True (Type: Delay). The product Smartphone with volume 5 was delivered to Lucknow after 2 days. SupplierName: Supplier_F with reliability score 92. Traffic condition: Normal. Reorder quantity: 87. Promotional events: Event_1. Anomaly detected: True (Type: Unusual Order Volume). The product Laptop with volume 98 was delivered to Jaipur after 2 days. SupplierName: Supplier_A with reliability score 71. Traffic condition: Congested. Reorder quantity: 64. Promotional events: Event_1. Anomaly detected: False (Type: Delay). The pro

CREATING AGENT

In [23]:
!pip install boto3
!pip install opensearch-py
!pip install botocore
!pip install awscli
!pip install retrying

In [94]:
import boto3
import json
import time
import zipfile
import logging
import pprint
from io import BytesIO

iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
dynamodb_client = boto3.client('dynamodb')
dynamodb_resource = boto3.resource('dynamodb')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)





def invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    end_session: bool = False
    if not session_state:
        session_state = {}

    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )

    if enable_trace:
        logger.info(pprint.pprint(agent_response))

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)


def create_agent_role(agent_name, agent_foundation_model, kb_id=None):
    agent_bedrock_allow_policy_name = f"{agent_name}-ba"
    agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
    # Create IAM policies for agent
    statements = [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{agent_foundation_model}"
            ]
        }
    ]
    # add Knowledge Base retrieve and retrieve and generate permissions if agent has KB attached to it
    if kb_id:
        statements.append(
            {
                "Sid": "QueryKB",
                "Effect": "Allow",
                "Action": [
                    "bedrock:Retrieve",
                    "bedrock:RetrieveAndGenerate"
                ],
                "Resource": [
                    f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{kb_id}"
                ]
            }
        )

    bedrock_agent_bedrock_allow_policy_statement = {
        "Version": "2012-10-17",
        "Statement": statements
    }

    bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)
    try:
        agent_bedrock_policy = iam_client.create_policy(
            PolicyName=agent_bedrock_allow_policy_name,
            PolicyDocument=bedrock_policy_json
        )
    except iam_client.exceptions.EntityAlreadyExistsException:
        agent_bedrock_policy = iam_client.get_policy(
            PolicyArn=f"arn:aws:iam::{account_id}:policy/{agent_bedrock_allow_policy_name}"
        )
                    
    # Create IAM Role for the agent and attach IAM policies
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)
    try:
        agent_role = iam_client.create_role(
            RoleName=agent_role_name,
            AssumeRolePolicyDocument=assume_role_policy_document_json
        )

        # Pause to make sure role is created
        time.sleep(10)
    except iam_client.exceptions.EntityAlreadyExistsException:
        agent_role = iam_client.get_role(
            RoleName=agent_role_name,
        )

    iam_client.attach_role_policy(
        RoleName=agent_role_name,
        PolicyArn=agent_bedrock_policy['Policy']['Arn']
    )
    return agent_role


def delete_agent_roles_and_policies(agent_name, kb_policy_name):
    agent_bedrock_allow_policy_name = f"{agent_name}-ba"
    agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
    dynamodb_access_policy_name = f'{agent_name}-dynamodb-policy'
    lambda_function_role = f'{agent_name}-lambda-role'

    for policy in [agent_bedrock_allow_policy_name, kb_policy_name]:
        try:
            iam_client.detach_role_policy(
                RoleName=agent_role_name,
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not detach {policy} from {agent_role_name}")
            print(e)

    for policy in [dynamodb_access_policy_name]:
        try:
            iam_client.detach_role_policy(
                RoleName=lambda_function_role,
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not detach {policy} from {lambda_function_role}")
            print(e)

    try:
        iam_client.detach_role_policy(
            RoleName=lambda_function_role,
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )
    except Exception as e:
        print(f"Could not detach AWSLambdaBasicExecutionRole from {lambda_function_role}")
        print(e)

    for role_name in [agent_role_name, lambda_function_role]:
        try:
            iam_client.delete_role(
                RoleName=role_name
            )
        except Exception as e:
            print(f"Could not delete role {role_name}")
            print(e)

    for policy in [agent_bedrock_allow_policy_name, kb_policy_name, dynamodb_access_policy_name]:
        try:
            iam_client.delete_policy(
                PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}'
            )
        except Exception as e:
            print(f"Could not delete policy {policy}")
            print(e)


def clean_up_resources(
        table_name, lambda_function, lambda_function_name, agent_action_group_response, agent_functions,
        agent_id, kb_id, alias_id
):
    action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
    action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']
    # Delete Agent Action Group, Agent Alias, and Agent
    try:
        bedrock_agent_client.update_agent_action_group(
            agentId=agent_id,
            agentVersion='DRAFT',
            actionGroupId= action_group_id,
            actionGroupName=action_group_name,
            actionGroupExecutor={
                'lambda': lambda_function['FunctionArn']
            },
            functionSchema={
                'functions': agent_functions
            },
            actionGroupState='DISABLED',
        )
        bedrock_agent_client.disassociate_agent_knowledge_base(
            agentId=agent_id,
            agentVersion='DRAFT',
            knowledgeBaseId=kb_id
        )
        bedrock_agent_client.delete_agent_action_group(
            agentId=agent_id,
            agentVersion='DRAFT',
            actionGroupId=action_group_id
        )
        bedrock_agent_client.delete_agent_alias(
            agentAliasId=alias_id,
            agentId=agent_id
        )
        bedrock_agent_client.delete_agent(agentId=agent_id)
        print(f"Agent {agent_id}, Agent Alias {alias_id}, and Action Group have been deleted.")
    except Exception as e:
        print(f"Error deleting Agent resources: {e}")

    # Delete Lambda function
    try:
        lambda_client.delete_function(FunctionName=lambda_function_name)
        print(f"Lambda function {lambda_function_name} has been deleted.")
    except Exception as e:
        print(f"Error deleting Lambda function {lambda_function_name}: {e}")

    # Delete DynamoDB table
    try:
        dynamodb_client.delete_table(TableName=table_name)
        print(f"Table {table_name} is being deleted...")
        waiter = dynamodb_client.get_waiter('table_not_exists')
        waiter.wait(TableName=table_name)
        print(f"Table {table_name} has been deleted.")
    except Exception as e:
        print(f"Error deleting table {table_name}: {e}")

In [95]:
import time
import boto3
import logging
import ipywidgets as widgets
import uuid




In [96]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-east-1', '199328835594')

In [59]:
suffix = f"{region}-{account_id}"
agent_name = 'anomaly-detect-awshack-1'
agent_bedrock_allow_policy_name = f"{agent_name}-ba"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

agent_description = "Agent in charge of going through complete prodcut and supplier data and then figures out anomalies in particular supplier deliveries"
agent_instruction = """
You are a supply chain management agent. you go through the  complete product and supplier data and figure out the anomaly reason in that supplier deliveries. You also provide solutions on how to resolve this supplier delivery anomaly.
"""

agent_action_group_description = """
Detectinganomaly"""    

agent_action_group_name = "Anomaly-detection-2"

In [60]:
agent_foundation_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
agent_foundation_model

'anthropic.claude-3-sonnet-20240229-v1:0'

In [61]:
agent_role = create_agent_role(agent_name, agent_foundation_model)

In [62]:
agent_role

{'Role': {'Path': '/',
  'RoleName': 'AmazonBedrockExecutionRoleForAgents_anomaly-detect-awshack-1',
  'RoleId': 'AROAS42HMVAFCJHFZHDBB',
  'Arn': 'arn:aws:iam::199328835594:role/AmazonBedrockExecutionRoleForAgents_anomaly-detect-awshack-1',
  'CreateDate': datetime.datetime(2024, 8, 27, 5, 14, 36, tzinfo=tzlocal()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Effect': 'Allow',
     'Principal': {'Service': 'bedrock.amazonaws.com'},
     'Action': 'sts:AssumeRole'}]}},
 'ResponseMetadata': {'RequestId': '12e02f21-1bf4-43ee-a3c9-1c423cf510cc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 27 Aug 2024 05:14:36 GMT',
   'x-amzn-requestid': '12e02f21-1bf4-43ee-a3c9-1c423cf510cc',
   'content-type': 'text/xml',
   'content-length': '875'},
  'RetryAttempts': 0}}

In [63]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=agent_foundation_model,
    instruction=agent_instruction,
)
response

{'ResponseMetadata': {'RequestId': 'f58025b8-2faf-430c-9c3a-b8da7d023aa6',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Tue, 27 Aug 2024 05:14:46 GMT',
   'content-type': 'application/json',
   'content-length': '870',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'f58025b8-2faf-430c-9c3a-b8da7d023aa6',
   'x-amz-apigw-id': 'dJwPFEofoAMELhg=',
   'x-amzn-trace-id': 'Root=1-66cd60c6-678f828e2889d9485a0b8ae3'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-east-1:199328835594:agent/JPLMXMHFQK',
  'agentId': 'JPLMXMHFQK',
  'agentName': 'anomaly-detect-awshack-1',
  'agentResourceRoleArn': 'arn:aws:iam::199328835594:role/AmazonBedrockExecutionRoleForAgents_anomaly-detect-awshack-1',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 8, 27, 5, 14, 46, 883743, tzinfo=tzlocal()),
  'description': 'Agent in charge of going through complete prodcut and supplier data and then figures out anomalies in particular supplier deliveries',
  'foundat

In [64]:
agent_id = response['agent']['agentId']
print("The agent id is:",agent_id)

The agent id is: JPLMXMHFQK


In [65]:
agent_functions = [
    
    {
        'name': 'create_Anomaly_solution',
        'description': 'figuring out anomalies in supplier delivery and then providing solutions to avoid them further',
        'parameters': {
            "SupplierName": {
                "description": "Name of the supplier",
                "required": True,
                "type": "string"
            },
            "Anomaly_detected": {
                "description": "The anomaly that is found after exploring the data of deliveriies by that supplier",
                "required": True,
                "type": "string"
            },
            "Anomaly_solution": {
                "description": "A insightful and good solution to avoid the anomaly in future deliveries",
                "required": True,
                "type": "string"
            }
        }
    },
    
]

In [66]:
# Pause to make sure agent is created
time.sleep(30)

# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda':'arn:aws:lambda:us-east-1:199328835594:function:anomaly_detection'
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [67]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '2df4c46b-ca5d-4ce2-8ed5-bd732679594f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 27 Aug 2024 05:15:17 GMT',
   'content-type': 'application/json',
   'content-length': '998',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2df4c46b-ca5d-4ce2-8ed5-bd732679594f',
   'x-amz-apigw-id': 'dJwT1ETZIAMEANA=',
   'x-amzn-trace-id': 'Root=1-66cd60e5-4430020946e3b0bd4bfdae73'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-east-1:199328835594:function:anomaly_detection'},
  'actionGroupId': 'TRJS81DBXP',
  'actionGroupName': 'Anomaly-detection-2',
  'actionGroupState': 'ENABLED',
  'agentId': 'JPLMXMHFQK',
  'agentVersion': 'DRAFT',
  'createdAt': datetime.datetime(2024, 8, 27, 5, 15, 17, 160166, tzinfo=tzlocal()),
  'description': '\nDetectinganomaly',
  'functionSchema': {'functions': [{'description': 'figuring out anomalies in supplier delivery and then providing solutions to avoid the

In [68]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')
try:
    response = lambda_client.add_permission(
        FunctionName= 'anomaly_detection',
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )
    print(response)
except Exception as e:
    print(e)

{'ResponseMetadata': {'RequestId': '302f01e0-b2c7-4942-82ed-a56d96806993', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Tue, 27 Aug 2024 05:15:17 GMT', 'content-type': 'application/json', 'content-length': '356', 'connection': 'keep-alive', 'x-amzn-requestid': '302f01e0-b2c7-4942-82ed-a56d96806993'}, 'RetryAttempts': 0}, 'Statement': '{"Sid":"allow_bedrock_JPLMXMHFQK","Effect":"Allow","Principal":{"Service":"bedrock.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:199328835594:function:anomaly_detection","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:bedrock:us-east-1:199328835594:agent/JPLMXMHFQK"}}}'}


In [69]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
print(response)
# Pause to make sure agent is prepared
time.sleep(30)

{'ResponseMetadata': {'RequestId': '46728b21-23bd-46ac-b980-08fafebf5725', 'HTTPStatusCode': 202, 'HTTPHeaders': {'date': 'Tue, 27 Aug 2024 05:15:17 GMT', 'content-type': 'application/json', 'content-length': '119', 'connection': 'keep-alive', 'x-amzn-requestid': '46728b21-23bd-46ac-b980-08fafebf5725', 'x-amz-apigw-id': 'dJwT4FH0IAMEpVQ=', 'x-amzn-trace-id': 'Root=1-66cd60e5-2bde58547eeddcfc378c4447'}, 'RetryAttempts': 0}, 'agentId': 'JPLMXMHFQK', 'agentStatus': 'PREPARING', 'agentVersion': 'DRAFT', 'preparedAt': datetime.datetime(2024, 8, 27, 5, 15, 17, 491200, tzinfo=tzlocal())}


In [104]:
alias_id = 'TSTALIASID'
AGENT_IG = 'JPLMXMHFQK'

INVOKING AND TESTING

In [113]:
%%time
session_id:str = str(uuid.uuid1())
query = "based on the data give the anomaly of Supplier_ERT in just a few words. Also give the solution on how to resolve the anomaly for further deliveries. If you are not able to find a supplier info say that the supplier does not exist " +summary 

response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

The supplier Supplier_ERT does not exist in the given data. Therefore, I cannot provide any anomaly or solution related to this supplier.
CPU times: user 19.1 ms, sys: 0 ns, total: 19.1 ms
Wall time: 7.41 s


TESTING

In [ ]:
import boto3
import json
import time
import zipfile
import logging
import pprint
from io import BytesIO

iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
dynamodb_client = boto3.client('dynamodb')
dynamodb_resource = boto3.resource('dynamodb')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)





def invoke_agent_helper(query, session_id, agent_id, alias_id, enable_trace=False, session_state=None):
    end_session: bool = False
    if not session_state:
        session_state = {}

    # invoke the agent API
    agent_response = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=alias_id,
        sessionId=session_id,
        enableTrace=enable_trace,
        endSession=end_session,
        sessionState=session_state
    )

    if enable_trace:
        logger.info(pprint.pprint(agent_response))

    event_stream = agent_response['completion']
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                if enable_trace:
                    logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                return agent_answer
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                if enable_trace:
                    logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [ ]:
#Clients
import time
import boto3
import logging
import ipywidgets as widgets
import uuid
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

In [91]:
alias_id = 'TSTALIASID'
AGENT_IG = 'JPLMXMHFQK'

In [92]:
%%time
session_id:str = str(uuid.uuid1())
query = "based on the data give the anomaly of Supplier_B in just a few words. Give a solution to the anomaly detected in a  few words" +summary 


response = invoke_agent_helper(query, session_id, agent_id, alias_id)
print(response)

I apologize, but I cannot provide any meaningful interpretation of the output you shared. As an AI assistant without direct access to the tool's internals, I can only speculate about the meaning of "{' SupplierID': '965072f9'}". However, speculating would go against the guidelines of providing reliable information to you. Could you please provide some context about what this output represents or clarify the original question? With more details, I can try to provide a more helpful response.
CPU times: user 20.7 ms, sys: 0 ns, total: 20.7 ms
Wall time: 17.9 s
